# Steel Surface Defect Classification Using LBP and GLCM

# Libraries and Modules

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import matplotlib.patches as patches

from skimage import io, color, img_as_ubyte
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Data Augmentation

Creating a new augmented dataset focused on regions of interest using data augmentation techniques and automatically saving it to files named according to the "defect_categories" list

In [3]:

# Define a list of defect categories
defect_categories = [
# mention defects List 
]

# Iterate through each defect category
for category in defect_categories:
    # Path to the 'Images' folder and 'Labels' folder for the current defect category
    images_folder = f'{category}_Images'
    labels_folder = f'{category}_Labels'
    
    # Create a folder for data augmentation if it doesn't exist
    data_aug_folder = category
    if not os.path.exists(data_aug_folder):
        os.makedirs(data_aug_folder)
    
    # Iterate through all image files in the 'Images' folder
    for image_filename in os.listdir(images_folder):
        if image_filename.endswith('.jpg'):
            # Load the image
            image_path = os.path.join(images_folder, image_filename)
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)

            # Find the corresponding XML file
            xml_filename = os.path.splitext(image_filename)[0] + '.xml'
            xml_path = os.path.join(labels_folder, xml_filename)

            # Parse the XML file
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Iterate through all objects in the XML file
            for i, obj in enumerate(root.findall('.//object')):
                defect_type = obj.find('name').text
                xmin = int(obj.find('bndbox/xmin').text)
                ymin = int(obj.find('bndbox/ymin').text)
                xmax = int(obj.find('bndbox/xmax').text)
                ymax = int(obj.find('bndbox/ymax').text)

                # Crop the region of interest from the image
                roi = image[ymin:ymax, xmin:xmax]

                # Save the original defect region
                original_filename = f"{image_filename}_{defect_type}_{i}_original.jpg"
                original_path = os.path.join(data_aug_folder, original_filename)
                cv2.imwrite(original_path, roi)

                # Perform data augmentation
                # Rotation
                rotated_filename = f"{image_filename}_{defect_type}_{i}_rotated.jpg"
                rotated_path = os.path.join(data_aug_folder, rotated_filename)
                rotated_roi = cv2.rotate(roi, cv2.ROTATE_90_CLOCKWISE)
                cv2.imwrite(rotated_path, rotated_roi)

                # Horizontal flipping
                hflipped_filename = f"{image_filename}_{defect_type}_{i}_hflipped.jpg"
                hflipped_path = os.path.join(data_aug_folder, hflipped_filename)
                hflipped_roi = cv2.flip(roi, 1)
                cv2.imwrite(hflipped_path, hflipped_roi)

                # Vertical flipping
                vflipped_filename = f"{image_filename}_{defect_type}_{i}_vflipped.jpg"
                vflipped_path = os.path.join(data_aug_folder, vflipped_filename)
                vflipped_roi = cv2.flip(roi, 0)
                cv2.imwrite(vflipped_path, vflipped_roi)


# Checking Data/Augmented Images for Each Defect

In [ ]:
# Define a list of defect categories
defect_categories = [
    # mention the defect list
]

# Iterate through each defect category
for category in defect_categories:
    # Path to the folder for the current defect category
    data_aug_folder = category
    
    # Count the number of augmented images
    num_augmented_images = len([f for f in os.listdir(data_aug_folder) if f.endswith('.jpg')])
    
    print(f"Defect Category: {category}")
    print(f"Number of Augmented Images: {num_augmented_images}\n")
